In [2]:
from galpy.potential import MiyamotoNagaiPotential, NFWPotential, HernquistPotential
from galpy.actionAngle import estimateDeltaStaeckel, actionAngleStaeckel
from galpy.actionAngle import UnboundError

from areposnap.gadget import gadget_readsnap
from areposnap.gadget_subfind import load_subfind

from auriga_basics import *
from auriga_functions import *

import numpy as np
from scipy import stats
from IPython.display import Image, display

import corner

import datetime
import random

import copy

from mpl_toolkits.axes_grid1 import make_axes_locatable
import matplotlib.pyplot as plt
import matplotlib
from matplotlib.patches import Circle
from matplotlib import animation
from matplotlib.colors import LogNorm


import datetime

cmap = copy.copy(plt.cm.inferno)
cmap.set_bad((0,0,0))

%matplotlib inline

In [3]:
# Simulation relevant stuff
machine = 'mac'
machine = 'magny'

if machine == 'magny':
    filedir = "/home/extmilan/masterthesis/files/"
    basedir = "/hits/universe/GigaGalaxy/level4_MHD/"
    plotdir = "/home/extmilan/masterthesis/plots/"
elif machine == 'mac':
    filedir = "/Users/smilanov/Documents/masterthesis/auriga_files/files/"
    basedir = "/Users/smilanov/Desktop/Auriga/level4/"
    plotdir = "/Users/smilanov/Documents/masterthesis/auriga_files/plots/"
else:
    raise NotADirectoryError
    
#### path = /hits/universe/GigaGalaxy/level4_MHD/halo_24/output/*
level = 4

for halo_number in [24]:  # range(1, 31):
    halodir = basedir+"halo_{0}/".format(halo_number)
    snappath = halodir+"output/"

    for snapnr in range(127,128,1):
        print("level   : {0}".format(level))
        print("halo    : {0}".format(halo_number))
        print("snapnr  : {0}".format(snapnr))
        print("basedir : {0}".format(basedir))
        print("halodir : {0}".format(halodir))
        print("snappath: {0}\n".format(snappath))
        s, sf = eat_snap_and_fof(level, halo_number, snapnr, snappath, loadonlytype=[4], 
            haloid=0, galradfac=0.1, verbose=True) 

        # Clean negative and zero values of gmet to avoid RuntimeErrors
        # later on (e.g. dividing by zero)
        s.data['gmet'] = np.maximum( s.data['gmet'], 1e-40 )
        

level   : 4
halo    : 24
snapnr  : 127
basedir : /hits/universe/GigaGalaxy/level4_MHD/
halodir : /hits/universe/GigaGalaxy/level4_MHD/halo_24/
snappath: /hits/universe/GigaGalaxy/level4_MHD/halo_24/output/

[ 34.42281723  33.16259384  37.29567337]
Found 1783163 stars.
Rotated pos.
Rotated vel.

galrad  : 0.02408556640148163
redshift: 2.220446049250313e-16
time    : 0.9999999999999998
center  : [ 0.  0.  0.]



In [4]:
"""
Snapshot =  127
v0_tot_kms at R0_kpc = 8.02852213383 : 220.724646624
Stellar Disk - a_MND_kpc = 2.96507719743 b_MND_kpc = 1.63627757204 v0_MND_kms = 105.005287928
Stellar Spheroid - a_HB_kp c= 1.71545528287 v0_HB_kms = 111.241910552
DM Halo - a_NFWH_kpc = 26.0152749345 v0_NFWH_kms = 159.117869741
"""

fix_v0_kms      = 220.724646624
fix_R0_kpc      = 8.02852213383
fix_a_MND_kpc   = 2.96507719743
fix_b_MND_kpc   = 1.63627757204
fix_v0_MND_kms  = 105.005287928
fix_a_HB_kpc    = 1.71545528287
fix_v0_HB_kms   = 111.241910552
fix_v0_NFWH_kms = 159.117869741
init_a_NFWH_kpc = 26.0152749345

fix_n_MND       = fix_v0_MND_kms**2  / fix_v0_kms**2
fix_n_HB        = fix_v0_HB_kms**2   / fix_v0_kms**2
fix_n_NFWH      = fix_v0_NFWH_kms**2 / fix_v0_kms**2

#_____function that sets-up galpy potential_____
def setup_galpy_potential(a_MND_kpc, b_MND_kpc, a_NFWH_kpc, a_HB_kpc, n_MND, n_NFWH, n_HB, _REFR0_kpc):
    
    #test input:
    if (a_MND_kpc <= 0.) or (b_MND_kpc <= 0.) or (a_NFWH_kpc <= 0.) or (a_HB_kpc <= 0.) \
       or (n_MND <= 0.) or (n_NFWH <= 0.) or (n_HB <= 0.) or (n_MND >= 1.) or (n_NFWH >= 1.) or (n_HB >= 1.):
        raise ValueError('Error in setup_galpy_potential: '+\
                         'The input parameters for the scaling profiles do not correspond to a physical potential.')
    if np.fabs(n_MND + n_NFWH + n_HB - 1.) > 1e-7:
        raise ValueError('Error in setup_galpy_potential: '+\
                         'The sum of the normalization does not add up to 1.')
        
    #trafo to galpy units:
    a_MND  = a_MND_kpc  / _REFR0_kpc
    b_MND  = b_MND_kpc  / _REFR0_kpc
    a_NFWH = a_NFWH_kpc / _REFR0_kpc
    a_HB   = a_HB_kpc   / _REFR0_kpc
    
    #setup potential:
    disk = MiyamotoNagaiPotential(
                a = a_MND,
                b = b_MND,
                normalize = n_MND)
    halo = NFWPotential(
                a = a_NFWH,
                normalize = n_NFWH)
    bulge = HernquistPotential(
                a = a_HB,
                normalize = n_HB) 
     
    return [disk,halo,bulge]


#for a in range(50):
    

In [5]:
# load GC IDs & create their mask
IDs = np.loadtxt(filedir + 'surviving_ids_snapshot_67_sh_1.txt')
gcmask = np.isin(s.id, IDs)


In [6]:
### for "true" a_NFWH
# set up galpy potential with values of potential fitting (either smoothed or not, depends on commenting out or not)
pot_galpy = setup_galpy_potential(fix_a_MND_kpc, fix_b_MND_kpc, init_a_NFWH_kpc, fix_a_HB_kpc, fix_n_MND, fix_n_NFWH, fix_n_HB, fix_R0_kpc)

# this would mean that there are no merged GCs
if np.sum(gcmask) == 0:
    print("skipped this step")
    #continue

# get position and velocities of all selected GCs & convert to galpy units
(R_kpc, phi_rad, z_kpc), (vR_kms, vphi_kms, vz_kms) = get_cylindrical_vectors(s, sf, gcmask)
# convert physical to galpy units by dividing by REF vals (get velocities from best fit parameters)
R_galpy, vR_galpy, vT_galpy, z_galpy, vz_galpy = R_kpc / fix_R0_kpc, vR_kms / fix_v0_kms, vphi_kms / fix_v0_kms, z_kpc / fix_R0_kpc, vz_kms / fix_v0_kms

# estimate Delta of the Staeckel potential
delta = 0.45
delta = estimateDeltaStaeckel(pot_galpy, R_galpy, z_galpy)
# CHECK HOW BIG INFLUENCE OF DELTA IS


# set up the actionAngleStaeckel object
aAS = actionAngleStaeckel(
        pot   = pot_galpy,  # potential
        delta = delta,      # focal length of confocal coordinate system
        c     = True        # use C code (for speed)
        )


jR_galpy, lz_galpy, jz_galpy, r_kpc = np.zeros(len(IDs)), np.zeros(len(IDs)), np.zeros(len(IDs)), np.zeros(len(IDs))
savedIDs = np.zeros(len(IDs))
IDs_notworking = []
for test_i, item in enumerate(IDs):
    if (test_i % 1000) == 0:
        print(datetime.datetime.now().time())
        print(test_i)
    try: 
        jR_galpy[test_i], lz_galpy[test_i], jz_galpy[test_i] = aAS(R_galpy[test_i], vR_galpy[test_i], vT_galpy[test_i], z_galpy[test_i], vz_galpy[test_i])
        r_kpc[test_i] = np.sqrt(R_kpc[test_i]**2 + z_kpc[test_i]**2)
        savedIDs[test_i] = item
    except(ValueError, UnboundError):
        IDs_notworking.append(item)
        continue
print('numbers of GCs wo actions:', len(IDs_notworking))
jR_kpckms, lz_kpckms, jz_kpckms = jR_galpy * fix_R0_kpc * fix_v0_kms, lz_galpy * fix_R0_kpc * fix_v0_kms, jz_galpy * fix_R0_kpc * fix_v0_kms

# just pick result values of particles of which I actually could calculate actions
survivor_id_mask = np.isin(IDs, savedIDs)
jR_kpckms, lz_kpckms, jz_kpckms = jR_kpckms[survivor_id_mask], lz_kpckms[survivor_id_mask], jz_kpckms[survivor_id_mask]
r_kpc = r_kpc[survivor_id_mask]
survivor_IDs = IDs[survivor_id_mask]
init_jR_kpckms = jR_kpckms
init_jz_kpckms = jz_kpckms

13:55:03.399179
0
13:55:51.083442
1000
13:56:40.021832
2000
numbers of GCs wo actions: 0


In [7]:
init_mean_jR_kpckms = np.mean(jR_kpckms)
init_std_jR_kpckms  = np.std(jR_kpckms)
init_skew_jR_kpckms = stats.skew(jR_kpckms)
init_mean_jz_kpckms = np.mean(jz_kpckms)
init_std_jz_kpckms  = np.std(jz_kpckms)
init_skew_jz_kpckms = stats.skew(jz_kpckms)
init_cov_jR_jz = np.cov(jR_kpckms, jz_kpckms)
init_mean_lz_kpckms = np.mean(lz_kpckms)


In [8]:
test_widths = np.arange(150,1050,50)
for test_width in test_widths:
    #test_width = 150.
    test_step = test_width / 2.

    cond_jR = np.where((jR_kpckms < (init_mean_jR_kpckms + test_step)) & (jR_kpckms > (init_mean_jR_kpckms - test_step)))
    cond_lz = np.where((lz_kpckms < (init_mean_lz_kpckms + test_step)) & (lz_kpckms > (init_mean_lz_kpckms - test_step)))
    cond_jz = np.where((jz_kpckms < (init_mean_jz_kpckms + test_step)) & (jz_kpckms > (init_mean_jz_kpckms - test_step)))


    jR_gcmask = np.isin(s.id, IDs[cond_jR])
    lz_gcmask = np.isin(s.id, IDs[cond_lz])
    jz_gcmask = np.isin(s.id, IDs[cond_jz])

    i_test_GCs = jR_gcmask * lz_gcmask * jz_gcmask


    print("number of GCs within test width = {} kpc/km/s: {}".format(test_width, np.sum(i_test_GCs)))

number of GCs within test width = 150 kpc/km/s: 0
number of GCs within test width = 200 kpc/km/s: 0
number of GCs within test width = 250 kpc/km/s: 0
number of GCs within test width = 300 kpc/km/s: 0
number of GCs within test width = 350 kpc/km/s: 0
number of GCs within test width = 400 kpc/km/s: 0
number of GCs within test width = 450 kpc/km/s: 0
number of GCs within test width = 500 kpc/km/s: 0
number of GCs within test width = 550 kpc/km/s: 0
number of GCs within test width = 600 kpc/km/s: 3
number of GCs within test width = 650 kpc/km/s: 3
number of GCs within test width = 700 kpc/km/s: 4
number of GCs within test width = 750 kpc/km/s: 4
number of GCs within test width = 800 kpc/km/s: 4
number of GCs within test width = 850 kpc/km/s: 6
number of GCs within test width = 900 kpc/km/s: 6
number of GCs within test width = 950 kpc/km/s: 7
number of GCs within test width = 1000 kpc/km/s: 11


In [9]:
### for "false" a_NFWH
#N = 50

from matplotlib.patches import Ellipse

def eigsorted(cov):
    vals, vecs = np.linalg.eigh(cov)
    order = vals.argsort()[::-1]
    return vals[order], vecs[:,order]

test_width = 1050.
test_step = test_width / 2.

cond_jR = np.where((jR_kpckms < (init_mean_jR_kpckms + test_step)) & (jR_kpckms > (init_mean_jR_kpckms - test_step)))
cond_lz = np.where((lz_kpckms < (init_mean_lz_kpckms + test_step)) & (lz_kpckms > (init_mean_lz_kpckms - test_step)))
cond_jz = np.where((jz_kpckms < (init_mean_jz_kpckms + test_step)) & (jz_kpckms > (init_mean_jz_kpckms - test_step)))


jR_gcmask = np.isin(s.id, IDs[cond_jR])
lz_gcmask = np.isin(s.id, IDs[cond_lz])
jz_gcmask = np.isin(s.id, IDs[cond_jz])

i_test_GCs = jR_gcmask * lz_gcmask * jz_gcmask

jR_lz_intersect = np.intersect1d(IDs[cond_jR], IDs[cond_lz])
jR_lz_jz_intersect = np.intersect1d(jR_lz_intersect, IDs[cond_jz])

num_GCs = np.sum(i_test_GCs)
used_IDs = jR_lz_jz_intersect

In [ ]:
startsnap = 67
endsnap = 127
actionmatrix = np.array([]) # should result in: action * GC * time
lookback_time_Gyr = []

jR_mean = []
lz_mean = []
jz_mean =[]
jR_std = []
lz_std =[]
jz_std = []

for snapnr in range(startsnap, endsnap + 1,1):
    i_snap = snapnr - startsnap 
    snapnnum, R0_kpc, v0_kms, a_MND_kpc, b_MND_kpc, a_HB_kpc, a_NFWH_kpc, v0_MND_kms, v0_HB_kms, v0_NFWH_kms = np.loadtxt(filedir + 'potential_params/pot_snap_' + str(snapnr))
    n_MND   = v0_MND_kms**2  / v0_kms**2
    n_HB    = v0_HB_kms**2   / v0_kms**2
    n_NFWH  = v0_NFWH_kms**2 / v0_kms**2

    fix_R0_kpc = R0_kpc
    fix_v0_kms = v0_kms

    print("load stars of snap {0}".format(snapnr))
    s, sf = eat_snap_and_fof(level, halo_number, snapnr, snappath, loadonlytype=[4], 
        haloid=0, galradfac=0.1, verbose=True) 

    # Clean negative and zero values of gmet to avoid RuntimeErrors
    # later on (e.g. dividing by zero)
    s.data['gmet'] = np.maximum( s.data['gmet'], 1e-40 )

    snap_time = s.cosmology_get_lookback_time_from_a( s.time, is_flat=True )
    lookback_time_Gyr.append(snap_time)

    try:
        pot_galpy = setup_galpy_potential(a_MND_kpc, b_MND_kpc, a_NFWH_kpc, a_HB_kpc, n_MND, n_NFWH, n_HB, R0_kpc)
    except:
        print('For snap {} no potential can be set up.'.format(int(snapnr)))
        filler1 = np.full(len(used_IDs), np.nan)
        filler2 = np.full(len(used_IDs), np.nan)
        actionmatrix = np.append(actionmatrix, [filler1, filler2, filler1])
        jR_mean.append(np.mean(np.nan))
        lz_mean.append(np.mean(np.nan))
        jz_mean.append(np.mean(np.nan))
        jR_std.append(np.std(np.nan))
        lz_std.append(np.std(np.nan))
        jz_std.append(np.std(np.nan))         
        continue


    i_test_GCs = np.isin(s.id, used_IDs)
    # get position and velocities of all selected GCs & convert to galpy units
    (R_kpc, phi_rad, z_kpc), (vR_kms, vphi_kms, vz_kms) = get_cylindrical_vectors(s, sf, i_test_GCs)
    # convert physical to galpy units by dividing by REF vals (get velocities from best fit parameters)
    R_galpy, vR_galpy, vT_galpy, z_galpy, vz_galpy = R_kpc / fix_R0_kpc, vR_kms / fix_v0_kms, vphi_kms / fix_v0_kms, z_kpc / fix_R0_kpc, vz_kms / fix_v0_kms

    # estimate Delta of the Staeckel potential
    delta = 0.45
    delta = estimateDeltaStaeckel(pot_galpy, R_galpy, z_galpy)
    # CHECK HOW BIG INFLUENCE OF DELTA IS


    # set up the actionAngleStaeckel object
    aAS = actionAngleStaeckel(
            pot   = pot_galpy,  # potential
            delta = delta,      # focal length of confocal coordinate system
            c     = True        # use C code (for speed)
            )


    jR_galpy, lz_galpy, jz_galpy, r_kpc = np.zeros(num_GCs), np.zeros(num_GCs), np.zeros(num_GCs), np.zeros(num_GCs)
    savedIDs = np.zeros(num_GCs)
    IDs_notworking = []


    for test_i, item in enumerate(used_IDs):
        '''        
        if (test_i % 1000) == 0:
            print(datetime.datetime.now().time())
            print(test_i)
        '''
        try: 
            jR_galpy[test_i], lz_galpy[test_i], jz_galpy[test_i] = aAS(R_galpy[test_i], vR_galpy[test_i], vT_galpy[test_i], z_galpy[test_i], vz_galpy[test_i])
            r_kpc[test_i] = np.sqrt(R_kpc[test_i]**2 + z_kpc[test_i]**2)
            savedIDs[test_i] = item
        except(ValueError, UnboundError):
            IDs_notworking.append(item)
            continue
    print('numbers of GCs wo actions:', len(IDs_notworking))
    jR_kpckms, lz_kpckms, jz_kpckms = jR_galpy * fix_R0_kpc * fix_v0_kms, lz_galpy * fix_R0_kpc * fix_v0_kms, jz_galpy * fix_R0_kpc * fix_v0_kms
    
    # just pick result values of particles of which I actually could calculate actions
    nonsurvivor_id_mask = np.isin(used_IDs, IDs_notworking)
    survivor_id_mask = np.isin(used_IDs, savedIDs)
    
    
    jR_kpckms = np.where(survivor_id_mask, jR_kpckms, np.nan)
    jz_kpckms = np.where(survivor_id_mask, jz_kpckms, np.nan)
    lz_kpckms = np.where(survivor_id_mask, lz_kpckms, np.nan)
    
    #jR_kpckms, lz_kpckms, jz_kpckms = jR_kpckms[survivor_id_mask], lz_kpckms[survivor_id_mask], jz_kpckms[survivor_id_mask]
    r_kpc = r_kpc[survivor_id_mask]
    print(jR_kpckms)
    survivor_IDs = used_IDs[survivor_id_mask]
    actionmatrix = np.append(actionmatrix, [jR_kpckms, lz_kpckms, jz_kpckms])
    jR_mean.append(np.nanmean(jR_kpckms))
    lz_mean.append(np.nanmean(lz_kpckms))
    jz_mean.append(np.nanmean(jz_kpckms))
    jR_std.append(np.nanstd(jR_kpckms))
    lz_std.append(np.nanstd(lz_kpckms))
    jz_std.append(np.nanstd(jz_kpckms))    
    ### TO DO:  if GC wo action add -99 values

load stars of snap 67
[ 13.4669323   12.98056126  13.84910297]
Found 371337 stars.
Rotated pos.
Rotated vel.

galrad  : 0.011243060231208801
redshift: 1.5312390291576135
time    : 0.395063440663206
center  : [ 0.  0.  0.]

numbers of GCs wo actions: 7
[             nan              nan   46304.03167256              nan
   26375.96492715  111968.36023207              nan              nan
    9316.38561406              nan   13020.55134688              nan
    5036.21907054    2476.37053578]
load stars of snap 68
[ 13.78952217  13.29696274  14.19396877]
Found 415479 stars.
Rotated pos.
Rotated vel.

galrad  : 0.011517290025949478
redshift: 1.4719748452657977
time    : 0.40453486082803386
center  : [ 0.  0.  0.]

For snap 68 no potential can be set up.
load stars of snap 69
[ 14.12320042  13.61969185  14.55005264]
Found 458329 stars.
Rotated pos.
Rotated vel.

galrad  : 0.011756277829408647
redshift: 1.4140982203725216
time    : 0.41423335287728646
center  : [ 0.  0.  0.]

numbers of GCs 

[ 20.81840324  20.10211563  21.86827469]
Found 1194051 stars.
Rotated pos.
Rotated vel.

galrad  : 0.017763620615005495
redshift: 0.6446418406845371
time    : 0.6080351206337894
center  : [ 0.  0.  0.]

numbers of GCs wo actions: 0
[ 1636.0396534   2182.6797602   5002.90841457  5924.01172806  9070.68497999
  4556.93027014  4300.67978426  4864.76188384  5988.49436694  6133.56307878
  3708.51965813  3338.48461924  4118.04390763  4289.58308896]
load stars of snap 90
[ 21.11937714  20.39230919  22.20462799]
Found 1227647 stars.
Rotated pos.
Rotated vel.

galrad  : 0.017964252829551698
redshift: 0.6214287452425136
time    : 0.616740021992414
center  : [ 0.  0.  0.]

numbers of GCs wo actions: 0
[  2243.72407813   1770.7477404    2720.37176733   2243.48231907
   4007.71288959   4940.59370277   7415.37146036   4035.97433326
   7202.75356429  16622.4118483    5190.94944664   3818.75030465
   4546.47900385   4735.77126047]
load stars of snap 91
[ 21.42389488  20.68611908  22.54693604]
Found 125

In [ ]:
print(actionmatrix)
print(np.shape(actionmatrix))

In [ ]:
len(jR_mean)

In [ ]:
finalmatrix= np.reshape(actionmatrix,(14,3,61), order='F')
jR_all = finalmatrix[:, 0]
lz_all = finalmatrix[:, 1]
jz_all = finalmatrix[:, 2]

In [ ]:
fig, ax = plt.subplots(3,1, sharex=True, figsize=(15,15))

num_GCs = len(jR_all)

for i in range(num_GCs):
    color = list(np.random.random( size=3))


    ax[0].plot(lookback_time_Gyr, jR_all[i], color = color, marker = '.', linestyle='None')
    ax[0].set_xlim(np.max(lookback_time_Gyr), np.min(lookback_time_Gyr))
    ax[0].set_ylim(0,15000)

    ax[1].plot(lookback_time_Gyr, lz_all[i], color = color, marker = '.', linestyle='None')
    ax[1].set_xlim(np.max(lookback_time_Gyr), np.min(lookback_time_Gyr))
    ax[1].set_ylim(-5000,5000)
    ax[2].plot(lookback_time_Gyr, jz_all[i], color = color, marker = '.', linestyle='None')
    ax[2].set_xlim(np.max(lookback_time_Gyr), np.min(lookback_time_Gyr))
    ax[2].set_ylim(0,None)

#ax[0].set_xlabel('lookback time [Gyr]')
#ax[1].set_xlabel('lookback time [Gyr]')
ax[2].set_xlabel('lookback time [Gyr]')

ax[0].plot(lookback_time_Gyr, jR_mean, 'k-')
ax[0].plot(lookback_time_Gyr, np.add(jR_mean, jR_std), 'r-', alpha = 0.5)
ax[0].plot(lookback_time_Gyr, np.subtract(jR_mean, jR_std), 'r-', alpha = 0.5)
ax[0].set_xlim(np.max(lookback_time_Gyr), np.min(lookback_time_Gyr))
ax[0].set_ylim(0,15000)

ax[1].plot(lookback_time_Gyr, lz_mean, 'k-')
ax[1].plot(lookback_time_Gyr, np.add(lz_mean, lz_std), 'r-', alpha = 0.5)
ax[1].plot(lookback_time_Gyr, np.subtract(lz_mean, lz_std), 'r-', alpha = 0.5)
ax[1].set_xlim(np.max(lookback_time_Gyr), np.min(lookback_time_Gyr))

ax[2].plot(lookback_time_Gyr, jz_mean, 'k-')
ax[2].plot(lookback_time_Gyr, np.add(jz_mean, jz_std), 'r-', alpha = 0.5)
ax[2].plot(lookback_time_Gyr, np.subtract(jz_mean, jz_std), 'r-', alpha = 0.5)

ax[0].set_ylabel('jR [kpc km/s]')
ax[1].set_ylabel('lz [kpc km/s]')
ax[2].set_ylabel('jz [kpc km/s]')
fig.tight_layout()

#ax[0].axvline(3.8, color = 'blue', linewidth=4, alpha =0.2)
#ax[1].axvline(3.8, color = 'blue', linewidth=4, alpha =0.2)
#ax[2].axvline(3.8, color = 'blue', linewidth=4, alpha =0.2)

In [ ]:
fig.savefig(plotdir + 'merger_67_small_box_action_evolution_with_errros.png', dpi = 300, format = 'png')

In [ ]:
for i in (wrong):
    break

In [ ]:
a_NFWH_arr = np.arange(10,41,1)
mean_jR_kpckms = []
std_jR_kpckms  = []
skew_jR_kpckms = []
mean_jz_kpckms = []
std_jz_kpckms  = []
skew_jz_kpckms = []
covs_jR_jz = []
for var_a_NFW_kpc in a_NFWH_arr:
    print("a_NFW = {}".format(var_a_NFW_kpc))
    # set up galpy potential with values of potential fitting (either smoothed or not, depends on commenting out or not)
    try:
        pot_galpy = setup_galpy_potential(fix_a_MND_kpc, fix_b_MND_kpc, var_a_NFW_kpc, fix_a_HB_kpc, fix_n_MND, fix_n_NFWH, fix_n_HB, fix_R0_kpc)
    except:
        mean_jR_kpckms.append(-99.)
        std_jR_kpckms.append(-99.)
        mean_lz_kpckms.append(-99.)
        std_lz_kpckms.append(-99.)
        mean_jz_kpckms.append(-99.)
        std_jz_kpckms.append(-99.)
        print('For a_NFW = {:d} no potential can be set up.'.format(var_a_NFW_kpc) )
        continue
    # this would mean that there are no merged GCs
    if np.sum(i_test_GCs) == 0:
        print("skipped this step")
        #continue

    # get position and velocities of all selected GCs & convert to galpy units
    (R_kpc, phi_rad, z_kpc), (vR_kms, vphi_kms, vz_kms) = get_cylindrical_vectors(s, sf, i_test_GCs)
    # convert physical to galpy units by dividing by REF vals (get velocities from best fit parameters)
    R_galpy, vR_galpy, vT_galpy, z_galpy, vz_galpy = R_kpc / fix_R0_kpc, vR_kms / fix_v0_kms, vphi_kms / fix_v0_kms, z_kpc / fix_R0_kpc, vz_kms / fix_v0_kms

    # estimate Delta of the Staeckel potential
    delta = 0.45
    delta = estimateDeltaStaeckel(pot_galpy, R_galpy, z_galpy)
    # CHECK HOW BIG INFLUENCE OF DELTA IS


    # set up the actionAngleStaeckel object
    aAS = actionAngleStaeckel(
            pot   = pot_galpy,  # potential
            delta = delta,      # focal length of confocal coordinate system
            c     = True        # use C code (for speed)
            )


    jR_galpy, lz_galpy, jz_galpy, r_kpc = np.zeros(num_GCs), np.zeros(num_GCs), np.zeros(num_GCs), np.zeros(num_GCs)
    savedIDs = np.zeros(num_GCs)
    IDs_notworking = []


    for test_i, item in enumerate(used_IDs):
        '''        
        if (test_i % 1000) == 0:
            print(datetime.datetime.now().time())
            print(test_i)
        '''
        try: 
            jR_galpy[test_i], lz_galpy[test_i], jz_galpy[test_i] = aAS(R_galpy[test_i], vR_galpy[test_i], vT_galpy[test_i], z_galpy[test_i], vz_galpy[test_i])
            r_kpc[test_i] = np.sqrt(R_kpc[test_i]**2 + z_kpc[test_i]**2)
            savedIDs[test_i] = item
        except(ValueError, UnboundError):
            IDs_notworking.append(item)
            continue
    print('numbers of GCs wo actions:', len(IDs_notworking))
    jR_kpckms, lz_kpckms, jz_kpckms = jR_galpy * fix_R0_kpc * fix_v0_kms, lz_galpy * fix_R0_kpc * fix_v0_kms, jz_galpy * fix_R0_kpc * fix_v0_kms

    # just pick result values of particles of which I actually could calculate actions
    survivor_id_mask = np.isin(used_IDs, savedIDs)
    jR_kpckms, lz_kpckms, jz_kpckms = jR_kpckms[survivor_id_mask], lz_kpckms[survivor_id_mask], jz_kpckms[survivor_id_mask]
    r_kpc = r_kpc[survivor_id_mask]
    survivor_IDs = used_IDs[survivor_id_mask]

    mean_jR_kpckms.append(np.mean(jR_kpckms))
    std_jR_kpckms.append(np.std(jR_kpckms))
    skew_jR_kpckms.append(stats.skew(jR_kpckms))
    mean_jz_kpckms.append(np.mean(jz_kpckms))
    std_jz_kpckms.append(np.std(jz_kpckms))
    skew_jz_kpckms.append(stats.skew(jz_kpckms))
    cov_jR_jz = np.cov(jR_kpckms, jz_kpckms)
    covs_jR_jz.append(cov_jR_jz)
    
    nstd = 2
    ax = plt.subplot(111)

    #cov = np.cov(x, y)
    vals, vecs = eigsorted(cov_jR_jz)
    theta = np.degrees(np.arctan2(*vecs[:,0][::-1]))
    w, h = 2 * nstd * np.sqrt(vals)
    ell = Ellipse(xy=(np.mean(jR_kpckms), np.mean(jz_kpckms)),
                  width=w, height=h,
                  angle=theta, color='black')
    ell.set_facecolor('none')
    ax.add_artist(ell)
    plt.hist2d(jR_kpckms, jz_kpckms, bins = 5)
    plt.xlabel('jR [kpc/km/s]')
    plt.ylabel('jz [kpc/km/s]')
    plt.show()

In [ ]:
# goal plot
fig, (ax1, ax2) = plt.subplots(2, 1, sharex=True, figsize = (15,10))

ax1.plot(a_NFWH_arr, np.array(skew_jR_kpckms), 'k.')
ax1.plot(init_a_NFWH_kpc, init_skew_jR_kpckms, marker = '+', color = 'red')
#ax1.xlabel('a$_{NFW}$')
ax1.set_yscale("log", nonposy='clip')
#ax1.hlines(init_mean_jR_kpckms, 0, np.max(a_NFWH_arr)+1, color = 'red', alpha = 0.5)
ax1.set_ylim(1, 10)

ax1.set_ylabel('skewness of j$_R$', size = 'x-large')
#plt.show()


ax2.plot(a_NFWH_arr, skew_jz_kpckms, 'k.')
ax2.plot(init_a_NFWH_kpc, init_skew_jz_kpckms, marker = '+', color = 'red')
#ax2.hlines(init_mean_jz_kpckms, 0, np.max(a_NFWH_arr)+1, color = 'red', alpha = 0.5)

ax2.set_xlabel('a$_{NFW}$ [kpc]', size = 'x-large')
#ax2.set_yscale("log", nonposy='clip')
ax2.set_ylim(1.615, 1.625)
ax2.set_ylabel('skewness of j$_z$', size = 'x-large')
#plt.show()

fig.tight_layout()

fig.savefig(plotdir + 'a_NFW_diagnostic_plot_skewness_small_box.png', dpi = 300, format = 'png')

In [ ]:
print(np.min(skew_jz_kpckms), np.max(skew_jz_kpckms))

In [ ]:
fig, (ax1, ax2) = plt.subplots(2, 1, sharex=True, figsize = (15,10))

ax1.plot(a_NFWH_arr, std_jR_kpckms, marker = '.', color = 'black', linestyle = None)
ax1.plot(init_a_NFWH_kpc, init_std_jR_kpckms, marker = '+', color = 'red')
#ax1.xlabel('a$_{NFW}$')
#ax1.set_yscale("log", nonposy='clip')
#ax1.hlines(init_mean_jR_kpckms, 0, np.max(a_NFWH_arr)+1, color = 'red', alpha = 0.5)
#ax1.set_ylim(10, None)

ax1.set_ylabel('std j$_R$ [kpc/km/s]', size = 'x-large')
#plt.show()


ax2.plot(a_NFWH_arr,std_jz_kpckms, marker = '.', color = 'black')
ax2.plot(init_a_NFWH_kpc, init_std_jz_kpckms, marker = '+', color = 'red')
#ax2.hlines(init_mean_jz_kpckms, 0, np.max(a_NFWH_arr)+1, color = 'red', alpha = 0.5)

ax2.set_xlabel('a$_{NFW}$ [kpc]', size = 'x-large')
#ax2.set_yscale("log", nonposy='clip')
#ax2.set_ylim(0, None)
ax2.set_ylabel('std j$_z$ [kpc/km/s]', size = 'x-large')
#plt.show()

fig.tight_layout()

fig.savefig(plotdir + 'a_67_NFW_diagnostic_plot_std_small_box.png', dpi = 300, format = 'png')

In [ ]:
fig, (ax1, ax2) = plt.subplots(2, 1, sharex=True, figsize = (15,10))

ax1.errorbar(a_NFWH_arr, np.array(mean_jR_kpckms), yerr = np.array(std_jR_kpckms), marker = '.', color = 'black', linestyle = None)
ax1.errorbar(init_a_NFWH_kpc, init_mean_jR_kpckms, init_std_jR_kpckms, marker = '+', color = 'red')
#ax1.xlabel('a$_{NFW}$')
ax1.set_yscale("log", nonposy='clip')
ax1.hlines(init_mean_jR_kpckms, 0, np.max(a_NFWH_arr)+1, color = 'red', alpha = 0.5)
ax1.set_ylim(10, None)

ax1.set_ylabel('mean j$_R$ [kpc/km/s]', size = 'x-large')
#plt.show()


ax2.errorbar(a_NFWH_arr, mean_jz_kpckms, yerr = std_jz_kpckms, marker = '.', color = 'black')
ax2.errorbar(init_a_NFWH_kpc, init_mean_jz_kpckms, init_std_jz_kpckms, marker = '+', color = 'red')
ax2.hlines(init_mean_jz_kpckms, 0, np.max(a_NFWH_arr)+1, color = 'red', alpha = 0.5)

ax2.set_xlabel('a$_{NFW}$ [kpc]', size = 'x-large')
#ax2.set_yscale("log", nonposy='clip')
ax2.set_ylim(0, None)
ax2.set_ylabel('mean j$_z$ [kpc/km/s]', size = 'x-large')
#plt.show()

fig.tight_layout()

fig.savefig(plotdir + 'a_NFW_diagnostic_plot_mean_std_small_box.png', dpi = 300, format = 'png')

In [ ]:
covs_jR_jz

In [ ]:
# try to show covariance matrix
for i in range(len(covs_jR_jz)):
    # plot correlation matrix
    fig = plt.figure()
    ax = fig.add_subplot(111)
    cax = ax.matshow(covs_jR_jz[i], norm = LogNorm())
    fig.colorbar(cax)
    ax.set_title('a_NFWH = {} kpc'.format(a_NFWH_arr[i]))
    plt.show()

In [ ]:


nstd = 2
ax = plt.subplot(111)

#cov = np.cov(x, y)
vals, vecs = eigsorted(covs_jR_jz[-1])
theta = np.degrees(np.arctan2(*vecs[:,0][::-1]))
w, h = 2 * nstd * np.sqrt(vals)
ell = Ellipse(xy=(mean_jR_kpckms[-1], mean_jz_kpckms[-1]),
              width=w, height=h,
              angle=theta, color='black')
ell.set_facecolor('none')
ax.add_artist(ell)
plt.hist2d(jR_kpckms, jz_kpckms, bins = 41)
plt.xlabel('jR [kpc/km/s]')
plt.ylabel('jz [kpc/km/s]')
plt.title('a_NFWH = {} kpc'.format(a_NFWH_arr[-1]))
plt.show()

In [ ]:
nstd = 2
ax = plt.subplot(111)

#cov = np.cov(x, y)
vals, vecs = eigsorted(init_cov_jR_jz)
theta = np.degrees(np.arctan2(*vecs[:,0][::-1]))
w, h = 2 * nstd * np.sqrt(vals)
ell = Ellipse(xy=(init_mean_jR_kpckms, init_mean_jz_kpckms),
              width=w, height=h,
              angle=theta, color='black')
ell.set_facecolor('none')
ax.add_artist(ell)
plt.hist2d(init_jR_kpckms, init_jz_kpckms, bins = 41)
plt.xlabel('jR [kpc/km/s]')
plt.ylabel('jz [kpc/km/s]')
plt.title('a_NFWH = {} kpc'.format(init_a_NFWH_kpc))
plt.show()

In [ ]:
# calculate concentration
M200 = sf.data['fmc2'][0] #M200
R200 = sf.data['frc2'][0] #R200
a = init_a_NFWH_kpc
c = a/R200

In [ ]:
c 